In [ ]:
!pip install openai chromadb langchain tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import GutenbergLoader

print('Python: ', platform.python_version())

Python:  3.9.16


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.environ["OPENAI_API_KEY"] = 'sk-**********************************'

In [ ]:
persist_directory="/content/drive/My Drive/Colab Notebooks/chroma/romeo"

In [ ]:
def get_gutenberg(url):
    loader = GutenbergLoader(url)
    data = loader.load()
    return data

In [ ]:
romeoandjuliet_data = get_gutenberg('https://www.gutenberg.org/cache/epub/1513/pg1513.txt')

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
romeoandjuliet_doc = text_splitter.split_documents(romeoandjuliet_data)

embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(romeoandjuliet_doc, embeddings, persist_directory=persist_directory)
vectordb.persist()

In [ ]:
romeoandjuliet_qa = ChatVectorDBChain.from_llm(OpenAI(temperature=0, model_name="gpt-3.5-turbo"), vectordb, return_source_documents=True)

/usr/local/lib/python3.9/dist-packages/langchain/llms/openai.py:170: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/langchain/llms/openai.py:624: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/langchain/chains/conversational_retrieval/base.py:191: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(


In [ ]:
chat_history = [("Romeo and Juliet","Romeo and Juliet")]

In [ ]:
query = "Have Romeo and Juliet spent the night together? Provide a verbose answer, referencing passages from the book."
result = romeoandjuliet_qa({"question": query, "chat_history": chat_history})

In [ ]:
result["source_documents"]

[Document(page_content='\n\n\n\r\n\n\nROMEO.\r\n\n\nYet banished? Hang up philosophy.\r\n\n\nUnless philosophy can make a Juliet,\r\n\n\nDisplant a town, reverse a Prince’s doom,\r\n\n\nIt helps not, it prevails not, talk no more.\r\n\n\n\r\n\n\nFRIAR LAWRENCE.\r\n\n\nO, then I see that mad men have no ears.\r\n\n\n\r\n\n\nROMEO.\r\n\n\nHow should they, when that wise men have no eyes?\r\n\n\n\r\n\n\nFRIAR LAWRENCE.\r\n\n\nLet me dispute with thee of thy estate.\r\n\n\n\r\n\n\nROMEO.\r\n\n\nThou canst not speak of that thou dost not feel.\r\n\n\nWert thou as young as I, Juliet thy love,\r\n\n\nAn hour but married, Tybalt murdered,\r\n\n\nDoting like me, and like me banished,\r\n\n\nThen mightst thou speak, then mightst thou tear thy hair,\r\n\n\nAnd fall upon the ground as I do now,\r\n\n\nTaking the measure of an unmade grave.\r\n\n\n\r\n\n\n [_Knocking within._]\r\n\n\n\r\n\n\nFRIAR LAWRENCE.\r\n\n\nArise; one knocks. Good Romeo, hide thyself.\r\n\n\n\r\n\n\nROMEO.\r\n\n\nNot I, unle

In [ ]:
result["answer"]

"There is no clear indication in the text that Romeo and Juliet spent the night together. In Act II, Scene 2, Romeo climbs over the Capulet's garden wall and enters the garden where he sees Juliet on her balcony. They exchange vows of love and plan to marry, but there is no mention of them spending the night together. Later in Act III, Scene 5, Juliet's mother urges her to marry Paris, and Juliet refuses, stating that she is already married to Romeo. This suggests that they did indeed marry, but again, there is no mention of them spending the night together. Overall, the text is ambiguous on this point and leaves it up to interpretation."